In [1]:
%reset

In [2]:
import sys

sys.path.insert(0, '/home/users/ids29/DGRB')

In [3]:
import aegis
import numpy as np
import torch
import healpy as hp
import pickle as pk
from astropy import units
from astropy import constants as c
import matplotlib.pyplot as plt
from os import listdir
import os
from sbi.inference import SNLE, SNPE#, prepare_for_sbi, simulate_for_sbi
from sbi import utils as utils
from sbi import analysis as analysis
# from sbi.inference.base import infer
from getdist import plots, MCSamples
from joblib import Parallel, delayed
from scipy.integrate import quad, simpson
import pickle
from scipy.stats import norm

%matplotlib inline

In [ ]:
grains=1000
num_simulations = 1000
num_workers = 48
delta = 1e-8

In [5]:
parameter_range_aegis = [[], []]
abundance_luminosity_and_spectrum_list = []
source_class_list = []
parameter_names = []
energy_range = [1000, 100000] #MeV 
energy_range_gen = [energy_range[0]*0.5, energy_range[1]*18]
max_radius = 8.5 + 20*2 #kpc
exposure = 2000*10*0.2 #cm^2 yr
flux_cut = 1e-9 #photons/cm^2/s
angular_cut = np.pi #10*u.deg.to('rad') #degrees
angular_cut_gen = np.pi #angular_cut*1.5
lat_cut = 0 #2*u.deg.to('rad') #degrees
lat_cut_gen = lat_cut*0.5

In [6]:
my_cosmology = 'Planck18'
z_range = [0, 14]
luminosity_range = 10.0**np.array([37, 50]) # Minimum value set by considering Andromeda distance using Fermi as benchmark and receiving 0.1 photon at detector side
my_AEGIS = aegis.aegis(abundance_luminosity_and_spectrum_list, source_class_list, parameter_range_aegis, energy_range, luminosity_range, max_radius, exposure, angular_cut, lat_cut, flux_cut, energy_range_gen=energy_range_gen, cosmology = my_cosmology, z_range = z_range, verbose = False)
my_AEGIS.angular_cut_gen, my_AEGIS.lat_cut_gen = angular_cut_gen, lat_cut_gen

In [7]:
def spec_poisson(energy, params):
    log_scaling_factor = params[0]
    scaling_factor = 10**log_scaling_factor
    Gamma = 2.2
    Emin, Emax = energy_range[0], energy_range[1]
    exposure_det = exposure*units.yr.to('s') # cm^2 s
    expected_photons = 70_000 # Value such that it produces the same number of photons (after mock_observe is applied) as the SFG#1 class at its Blanco & Linden's nominal value.
    num_photons_exposure_solidAngle = expected_photons / exposure_det / (4*np.pi) # photons/cm^2/sec/sr
    normalization =  (Emax**(1-Gamma) - Emin**(1-Gamma)) / (1-Gamma) 
    prop_const = num_photons_exposure_solidAngle / normalization
    return scaling_factor * prop_const * energy**(-Gamma)

In [8]:
als_Poisson = [spec_poisson]
my_AEGIS.abun_lum_spec = [als_Poisson]
my_AEGIS.source_class_list = ['isotropic_diffuse']

In [9]:
# a simple simulator with the total number of photons as the summary statistic
def simulator(params):

    input_params = params.numpy()

    source_info = my_AEGIS.create_sources(input_params, grains=grains, epsilon=1e-3)
    photon_info = my_AEGIS.generate_photons_from_sources(input_params, source_info, grains=grains) 
    obs_info = {'psf_fits_path': '/home/users/ids29/DGRB/FERMI_files/psf_P8R3_ULTRACLEANVETO_V2_PSF.fits', 'edisp_fits_path': '/home/users/ids29/DGRB/FERMI_files/edisp_P8R3_ULTRACLEANVETO_V2_PSF.fits', 'event_type': 'PSF3', 'exposure_map': None}
    obs_photon_info = my_AEGIS.mock_observe(photon_info, obs_info)
    
    return obs_photon_info

In [10]:
def manual_simulate_for_sbi(proposal, num_simulations=1000, num_workers=32):
    """
    Simulates the model in parallel using joblib.
    Each simulation call samples a parameter from the proposal and passes the index to the simulator.
    """
    def run_simulation(i):
        if i % 10 == 0:
            print(f"i= {i}")
        # Sample a parameter from the proposal (sbi.utils.BoxUniform has a .sample() method)
        theta_i = proposal.sample()
        photon_info = simulator(theta_i)
        return theta_i, photon_info

    # Run simulations in parallel using joblib.
    results = Parallel(n_jobs=num_workers, timeout=None)(delayed(run_simulation)(i) for i in range(num_simulations))
    theta_list, photon_info_list = zip(*results)

    theta_tensor = torch.stack(theta_list, dim=0).to(torch.float32)
    
    
    return theta_tensor, photon_info_list

In [11]:
# Define the prior using sbi.utils.BoxUniform
prior_range = torch.tensor([[-2],
                            [1]])

prior = utils.BoxUniform(low=prior_range[0], high=prior_range[1])

train_thetas, train_photon_info = manual_simulate_for_sbi(prior,
                                   num_simulations=num_simulations,
                                   num_workers=num_workers)

i= 30
i= 50
i= 40
i= 0
i= 10
i= 60
i= 70
i= 80
i= 20
i= 90
i= 100
i= 110
i= 120
i= 130
i= 140
i= 150
i= 160
i= 170
i= 180
i= 190
i= 200
i= 210
i= 220
i= 230
i= 240
i= 250
i= 260
i= 270
i= 280
i= 290
i= 300
i= 310
i= 320
i= 330
i= 340
i= 350
i= 360
i= 370
i= 380
i= 390
i= 400
i= 410
i= 420
i= 430
i= 440
i= 450
i= 460
i= 470
i= 480
i= 490
i= 500
i= 510
i= 520
i= 530
i= 540
i= 550
i= 560
i= 570
i= 580
i= 590
i= 600
i= 610
i= 620
i= 630
i= 640
i= 650
i= 660
i= 670
i= 680
i= 690
i= 700
i= 710
i= 720
i= 730
i= 740
i= 750
i= 760
i= 770
i= 780
i= 790
i= 800
i= 810
i= 820
i= 830
i= 840
i= 850
i= 860
i= 870
i= 880
i= 890
i= 900
i= 910
i= 920
i= 930
i= 940
i= 950
i= 960
i= 970
i= 980
i= 990


In [12]:
# 'photon_info_list' is a list of dictionaries

# Save to file
with open('train_data_Poisson.pkl', 'wb') as f:
    pickle.dump(train_photon_info, f)

# Save to file
torch.save(train_thetas, 'train_thetas_Poisson.pt')
torch.save(prior_range, 'prior_range_Poisson.pt')

In [16]:
test_theta = torch.tensor([np.log10(1)]) # A_Poisson = 1
test_photon_info = simulator(test_theta)
print(f"Number of photons after mock_observe: {test_photon_info['energies'].size}")

Number of photons after mock_observe: 32235


In [14]:
torch.save(test_theta, r'test_theta_Poisson.pt')

with open(r'test_data_Poisson.pkl', 'wb') as f:
    pickle.dump(test_photon_info, f)

Use the following value for the SFG + Poisson case

In [17]:
test_theta = torch.tensor([np.log10(0.5)]) # A_Poisson = 0.5
test_photon_info = simulator(test_theta)
print(f"Number of photons after mock_observe: {test_photon_info['energies'].size}")

Number of photons after mock_observe: 15890
